In [2]:
import csv
from datetime import datetime, timedelta
def partionnage(chemin):
    row=[]
    with open(chemin, 'r') as file:
        i=0
        reader = csv.reader(file, delimiter=';')
        new_row={}
        for row in reader:
            if(i!=0):
                temp_dic={}
                temp_dic["date"]=(row[0].split(","))[1].split(" ")[0]
                temp_dic["heure"]=int((row[0].split(","))[1].split(" ")[1].split(":")[0])
                temp_dic["OT"]=float((row[0].split(","))[2])
                new_row[int((row[0].split(","))[0])]=temp_dic
            i=i+1
        return new_row
    

data=partionnage('ETTh1_without_missing.csv')
data={cle: valeur for i, (cle, valeur) in enumerate(data.items()) if i > 8659}
# Récupérer la dernière valeur
last_entry = data[max(data.keys())]

# Convertir la date et l'heure en objets datetime
last_date = datetime.strptime(last_entry['date'], "%Y-%m-%d")
last_hour = last_entry['heure']
dataFinaleValue=[]
# Incrémenter la dernière date et l'heure 100 fois
for _ in range(100):
    next_date = last_date + timedelta(hours=last_hour)
    # Formater la date dans le même format que celui d'origine
    next_date_formatted = next_date.strftime("%Y-%m-%d %H:%M:%S")
    print(next_date_formatted)
    dataFinaleValue.append(next_date_formatted)
    # Incrémenter d'une heure
    last_hour += 1


2018-06-22 15:00:00
2018-06-22 16:00:00
2018-06-22 17:00:00
2018-06-22 18:00:00
2018-06-22 19:00:00
2018-06-22 20:00:00
2018-06-22 21:00:00
2018-06-22 22:00:00
2018-06-22 23:00:00
2018-06-23 00:00:00
2018-06-23 01:00:00
2018-06-23 02:00:00
2018-06-23 03:00:00
2018-06-23 04:00:00
2018-06-23 05:00:00
2018-06-23 06:00:00
2018-06-23 07:00:00
2018-06-23 08:00:00
2018-06-23 09:00:00
2018-06-23 10:00:00
2018-06-23 11:00:00
2018-06-23 12:00:00
2018-06-23 13:00:00
2018-06-23 14:00:00
2018-06-23 15:00:00
2018-06-23 16:00:00
2018-06-23 17:00:00
2018-06-23 18:00:00
2018-06-23 19:00:00
2018-06-23 20:00:00
2018-06-23 21:00:00
2018-06-23 22:00:00
2018-06-23 23:00:00
2018-06-24 00:00:00
2018-06-24 01:00:00
2018-06-24 02:00:00
2018-06-24 03:00:00
2018-06-24 04:00:00
2018-06-24 05:00:00
2018-06-24 06:00:00
2018-06-24 07:00:00
2018-06-24 08:00:00
2018-06-24 09:00:00
2018-06-24 10:00:00
2018-06-24 11:00:00
2018-06-24 12:00:00
2018-06-24 13:00:00
2018-06-24 14:00:00
2018-06-24 15:00:00
2018-06-24 16:00:00


In [16]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from datetime import datetime

#sans normalisation
def pretraitementNewValue(date, hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    return np.array([[years, months, days, hour]])

def predictionNeuronne(data, numberTry=10):
    mae = 1000
    maeFind = []
    for i in range(numberTry):
        dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
        years = [date.year for date in dates]
        months = [date.month for date in dates]
        days = [date.day for date in dates]
        hours = [value['heure'] for key, value in data.items()]
        ots = [value['OT'] for key, value in data.items()]
        
        features = np.column_stack((years, months, days, hours))
        
        X_train, X_test, y_train, y_test = train_test_split(features, ots, test_size=0.1, random_state=42)
        
        X_train, X_val, y_train, y_val = train_test_split(np.array(X_train), np.array(y_train), test_size=0.3, random_state=42)
        
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        model.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_val, np.array(y_val)))
        loss, Newmae = model.evaluate(X_test, np.array(y_test))
        print("mae=", Newmae)
        
        # Évaluation du modèle sur l'ensemble de test
        predictions = model.predict(X_test)
        maeFind.append(Newmae)
        
        # Prétraitement et prédiction pour une nouvelle valeur
        new_value_prediction = model.predict(pretraitementNewValue("2018-06-22", 10))
        print(new_value_prediction)
        
        if mae > Newmae:
            filename = "submissionReseauDeNeuronne.csv"
            headers = ["Id", "OT"]
            resp = []
            for i in range(100):
                liste = []
                print(dataFinaleValue[i].split(" ")[0])
                print(dataFinaleValue[i].split(" ")[1].split(":")[0])
                new_value_prediction = model.predict(pretraitementNewValue(dataFinaleValue[i].split(" ")[0], int(dataFinaleValue[i].split(" ")[1].split(":")[0])))
                liste = [i, float(new_value_prediction)]
                resp.append(liste)
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(headers)
                for row in resp:
                    writer.writerow(row)
            print(f"Fichier CSV '{filename}' créé avec succès.")
            mae = Newmae

#sans normalisation

predictionNeuronne(data)


Epoch 1/10
171/171 [==============================] - 1s 2ms/step - loss: 328.8820 - mae: 9.4906 - val_loss: 33.7558 - val_mae: 4.6937
Epoch 2/10
171/171 [==============================] - 0s 1ms/step - loss: 36.3726 - mae: 4.8326 - val_loss: 33.5215 - val_mae: 4.5784
Epoch 3/10
171/171 [==============================] - 0s 1ms/step - loss: 36.2087 - mae: 4.8288 - val_loss: 38.0079 - val_mae: 5.2041
Epoch 4/10
171/171 [==============================] - 0s 1ms/step - loss: 36.0168 - mae: 4.8144 - val_loss: 34.4207 - val_mae: 4.5726
Epoch 5/10
171/171 [==============================] - 0s 1ms/step - loss: 36.4266 - mae: 4.8497 - val_loss: 35.8582 - val_mae: 4.6091
Epoch 6/10
171/171 [==============================] - 0s 1ms/step - loss: 35.5266 - mae: 4.7890 - val_loss: 32.1990 - val_mae: 4.5740
Epoch 7/10
171/171 [==============================] - 0s 1ms/step - loss: 37.3372 - mae: 4.8852 - val_loss: 38.8109 - val_mae: 5.3089
Epoch 8/10
171/171 [==============================] - 0s 1ms/

In [ ]:
#en normalisant tout

def pretraitementNewValue(date, hour, scaler_year, scaler_month, scaler_day, scaler_hour):
    years = int(date.split("-")[0])
    months = int(date.split("-")[1])
    days = int(date.split("-")[2])
    hour_scaled = scaler_hour.transform([[hour]])
    return np.column_stack((scaler_year.transform([[years]]),
                            scaler_month.transform([[months]]),
                            scaler_day.transform([[days]]),
                            hour_scaled))
def predictionNeuronne(data, numberTry=10):
    mae=1000
    maeFind = []
    for i in range(numberTry):
        dates = [datetime.strptime(value['date'], '%Y-%m-%d') for key, value in data.items()]
        years = [date.year for date in dates]
        months = [date.month for date in dates]
        days = [date.day for date in dates]
        hours = [value['heure'] for key, value in data.items()]
        ots = [value['OT'] for key, value in data.items()]
        
        scaler_year = StandardScaler()
        scaler_month = StandardScaler()
        scaler_day = StandardScaler()
        scaler_hour = StandardScaler()
        scaler_ot = StandardScaler()
        
        years_scaled = scaler_year.fit_transform(np.array(years).reshape(-1, 1))
        months_scaled = scaler_month.fit_transform(np.array(months).reshape(-1, 1))
        days_scaled = scaler_day.fit_transform(np.array(days).reshape(-1, 1))
        hours_scaled = scaler_hour.fit_transform(np.array(hours).reshape(-1, 1))
        ots_scaled = scaler_ot.fit_transform(np.array(ots).reshape(-1, 1))
        
        features = np.column_stack((years_scaled, months_scaled, days_scaled, hours_scaled))
        
        X_train, X_test, y_train, y_test = train_test_split(features, ots_scaled, test_size=0.2, random_state=42)
        
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3)
        loss, Newmae = model.evaluate(X_test, y_test)
        print("mae=", Newmae)
        
        # Évaluation du modèle sur l'ensemble de test
        predictions_scaled = model.predict(X_test)
        maeFind.append(Newmae)
        
        # Prétraitement et prédiction pour une nouvelle valeur non normalisée
        new_value_features = pretraitementNewValue("2018-06-22", 10, scaler_year, scaler_month, scaler_day, scaler_hour)
        new_value_prediction_scaled = model.predict(new_value_features)
        new_value_prediction = scaler_ot.inverse_transform(new_value_prediction_scaled)
        print(new_value_prediction)
        if(mae>Newmae):
                filename="submissionReseauDeNeuronneNormalisee.csv"
                headers = ["Id","OT"]
                resp=[]
                for i in range(100):
                    liste=[]
                    print(dataFinaleValue[i].split(" ")[0])
                    print(dataFinaleValue[i].split(" ")[1].split(":")[0])
                    new_value_features = pretraitementNewValue(dataFinaleValue[i].split(" ")[0],int(dataFinaleValue[i].split(" ")[1].split(":")[0]), scaler_year, scaler_month, scaler_day, scaler_hour)
                    new_value_prediction_scaled = model.predict(new_value_features)
                    new_value_prediction = scaler_ot.inverse_transform(new_value_prediction_scaled)
                    liste=[i,float(new_value_prediction)]
                    resp.append(liste)
                with open(filename, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(headers)
                    for row in resp:
                        writer.writerow(row)
                print(f"Fichier CSV '{filename}' créé avec succès.")
                mae=Newmae
predictionNeuronne(data)





Epoch 1/10
152/152 [==============================] - 1s 2ms/step - loss: 0.4461 - mae: 0.5147 - val_loss: 0.1923 - val_mae: 0.3500
Epoch 2/10
152/152 [==============================] - 0s 2ms/step - loss: 0.1848 - mae: 0.3417 - val_loss: 0.1687 - val_mae: 0.3298
Epoch 3/10
152/152 [==============================] - 0s 2ms/step - loss: 0.1732 - mae: 0.3305 - val_loss: 0.1652 - val_mae: 0.3253
Epoch 4/10
152/152 [==============================] - 0s 2ms/step - loss: 0.1645 - mae: 0.3208 - val_loss: 0.1568 - val_mae: 0.3129
Epoch 5/10
152/152 [==============================] - 0s 1ms/step - loss: 0.1613 - mae: 0.3179 - val_loss: 0.1523 - val_mae: 0.3069
Epoch 6/10
152/152 [==============================] - 0s 1ms/step - loss: 0.1559 - mae: 0.3110 - val_loss: 0.1504 - val_mae: 0.3064
Epoch 7/10
152/152 [==============================] - 0s 2ms/step - loss: 0.1529 - mae: 0.3092 - val_loss: 0.1510 - val_mae: 0.3053
Epoch 8/10
152/152 [==============================] - 0s 2ms/step - loss: 0.